In [1]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
import pandas as pd
import numpy as np
# import LEAF modules.
import LEAF
import SL2PV0 
import SL2PV1

In [3]:
import toolsUtils
import eoImage
import toolsNets
import eoImage
import toolsUtils
import dictionariesSL2P 
from datetime import timedelta
from datetime import datetime

In [4]:
siteList=["projects/ee-modis250/assets/validationGBOV/NEON_2013-22_RMs_updated_parsed"]
imageCollectionName='COPERNICUS/S2_SR_HARMONIZED'
algorithm=SL2PV1 
variableName="LAI"
maxCloudcover=90
outputScaleSize=30
inputScaleSize=30
bufferSpatialSize = 45
bufferTemporalSize =[-7,7]
subsamplingFraction = 0.99
outputFileName='C:/Djamai_Najib/1_vegetation_parameters/1_scripts/LEAF-Landsat-Validation-paper/data/'
input=siteList[0]
collectionOptions = (dictionariesSL2P.make_collection_options(algorithm))
networkOptions= dictionariesSL2P.make_net_options()

In [5]:
sampleRecords =  ee.FeatureCollection(input).sort('system:time_start', False).map(lambda feature: feature.set('timeStart',feature.get('system:time_start')))
sampleRecords =  sampleRecords.toList(sampleRecords.size())
print('Site: ',input, ' with ',sampleRecords.size().getInfo(), ' features.')

Site:  projects/ee-modis250/assets/validationGBOV/NEON_2013-22_RMs_updated_parsed  with  14141  features.


In [6]:
n=0
site = ee.Feature(sampleRecords.get(n))
startDate = datetime.fromtimestamp(ee.Date.parse("dd/MM/YY",site.get('system:time_start'),'Etc/GMT+6').getInfo()['value']/1000) + timedelta(days=bufferTemporalSize[0])
endDate = datetime.fromtimestamp(ee.Date.parse("dd/MM/YY",site.get('system:time_end'),'Etc/GMT+6').getInfo()['value']/1000) + timedelta(days=bufferTemporalSize[1])
endDatePlusOne = endDate + timedelta(days=1)
dateRange = pd.DataFrame( {'startDate':[startDate],'endDate':[endDatePlusOne]})
index=0
Dates=dateRange.iloc[0]

In [7]:
colOptions=collectionOptions[imageCollectionName]
netOptions=networkOptions[variableName][imageCollectionName]

colOptions,netOptions,
variable=variableName
mapBounds=site.geometry()


products = []
tools = colOptions['tools']
    
numNets = ee.Number(ee.Feature((colOptions["Network_Ind"]).first()).propertyNames().remove('lon').remove('Feature Index').remove('system:index').size())
SL2P = ee.List.sequence(1,ee.Number(colOptions["numVariables"]),1).map(lambda netNum: toolsNets.makeNetVars(colOptions["Collection_SL2P"],numNets,netNum))
errorsSL2P = ee.List.sequence(1,ee.Number(colOptions["numVariables"]),1).map(lambda netNum: toolsNets.makeNetVars(colOptions["Collection_SL2Perrors"],numNets,netNum))
len(SL2P.getInfo()),len(errorsSL2P.getInfo()),len(SL2P.getInfo()[0]),len(errorsSL2P.getInfo()[0])

(7, 7, 12, 12)

In [8]:
def makeIndexLayer(image, legend, Network_Ind):
    image = ee.Image(image)                          # partition image
    legend = ee.FeatureCollection(legend)            # legend to convert partition numbers to networks
    Network_Ind = ee.FeatureCollection(Network_Ind)  # legend to convert networks to networkIDs
    
    # get lists of valid partitions
    legend_list = legend.toList(legend.size())
    landcover = legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

    # get corresponding networkIDs
    networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
                                    .map(lambda propertyValue: ee.Feature(ee.FeatureCollection(Network_Ind).first()) \
                                    .toDictionary().getNumber(propertyValue))
    
    return image.remap(landcover, networkIDs, 0).rename('networkID')

In [9]:
def selectNet(image, netList, inputNames, netIndex):
    
    image = ee.Image(image)
    netList = ee.List(netList)
    inputNames = ee.List(inputNames)
    netIndex = ee.Number(netIndex).int()
    
    return ee.Dictionary()  \
            .set("Image", ee.Image(image.updateMask(image.select('networkID').eq(netIndex)).select(inputNames))) \
            .set("Network", ee.List(netList.get(netIndex)))

In [10]:
def applyNet(outputName, netDict):
    outputName = ee.String(outputName)
    netDict = ee.Dictionary(netDict)
    inp = ee.Image(netDict.get('Image'))
    net = ee.Dictionary(netDict.get('Network'))
    
    # input scaling
    l1inp2D = inp.multiply(ee.Image(net.toArray(ee.List(['inpSlope']),0).transpose()).arrayProject([0]) \
                        .arrayFlatten([inp.bandNames()])) \
                        .add(ee.Image(net.toArray(ee.List(['inpOffset']),0).transpose()) \
                        .arrayProject([0]).arrayFlatten([inp.bandNames()]))
    
    # hidden layers
    l12D = ee.Image(net.toArray(ee.List(['h1wt']),0).reshape([ee.List(net.get('h1bi')).length(),ee.List(net.get('inpOffset')).length()])) \
              .matrixMultiply(l1inp2D.toArray().toArray(1)) \
              .add(ee.Image(net.toArray(ee.List(['h1bi']),0).transpose())) \
              .arrayProject([0]).arrayFlatten([['h1w1','h1w2','h1w3','h1w4','h1w5']])
    
    # apply tansig 2/(1+exp(-2*n))-1
    l2inp2D = ee.Image(2).divide(ee.Image(1).add((ee.Image(-2).multiply(l12D)).exp())).subtract(ee.Image(1))
    
    # purlin hidden layers
    l22D = l2inp2D.multiply(ee.Image(net.toArray(ee.List(['h2wt']),0).transpose()) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2w1','h2w2','h2w3','h2w4','h2w5']])) \
                    .reduce('sum') \
                    .add(ee.Image(net.toArray(ee.List(['h2bi']),0))) \
                                          .arrayProject([0]) \
                                          .arrayFlatten([['h2bi']])
    
    # output scaling 
    outputBand = l22D.subtract(ee.Image(ee.Number(net.get('outBias')))).divide(ee.Image(ee.Number(net.get('outSlope')))) 
    
    # return network output
    return (outputBand.rename(outputName))

In [11]:
def wrapperNNets(network, partition, netOptions, colOptions, suffixName, outputName, imageInput):
    # typecast function parameters
    network = ee.List(network)
    partition = ee.Image(partition)
    netOptions = netOptions
    colOptions = colOptions
    suffixName = suffixName
    imageInput = ee.Image(imageInput)
    outputName = outputName
    
    # parse partition  used to identify network to use
    partition = partition.clip(imageInput.geometry()).select(['partition'])

    # determine networks based on collection
    netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1)))

    # parse land cover into network index and add to input image
    imageInput = imageInput.addBands(makeIndexLayer(partition,colOptions["legend"],colOptions["Network_Ind"]))

    # define list of input names
    return netList,imageInput,ee.ImageCollection(ee.List.sequence(0, netList.size().subtract(1)) \
                                                    .map(lambda netIndex: selectNet(imageInput,netList,netOptions["inputBands"],netIndex)) \
                                                    .map(lambda netDict: applyNet(suffixName+outputName,netDict))) \
                                                    .max().addBands(partition).addBands(imageInput.select('networkID'))

In [12]:
L=colOptions["legend"].getInfo()

In [13]:
L['features'][0]['properties']

{'B': 0,
 'B (hex)': '0',
 'Description': 'Temperate or sub-polar needleleaf forest',
 'Earth Engine Palette': "'003D00' , // Temperate or sub-polar needleleaf forest",
 'G': 61,
 'G (hex)': '3D',
 'Lon': 0,
 'R': 0,
 'R (hex)': '0',
 'SL2P Network': 'evergreen_needleaf_forest',
 'Value': 1}

In [14]:
dict={}
for ii in L['features']:
    dict.update({ii['properties']['Value']:ii['properties']['SL2P Network']})

In [15]:
dict

{1: 'evergreen_needleaf_forest',
 3: 'deciduous_broadleaf_forest',
 4: 'deciduous_broadleaf_forest',
 17: 'invalid',
 7: 'shrublands',
 8: 'polar_shrubland',
 5: 'deciduous_broadleaf_forest',
 11: 'polar_shrubland',
 9: 'grassland_pasture',
 15: 'close_cropland',
 13: 'lichen_feathermoss',
 2: 'evergreen_needleaf_forest',
 14: 'grassland_pasture',
 10: 'polar_grassland',
 12: 'sphagnum_feathermoss',
 18: 'invalid',
 16: 'sparse_cropland',
 19: 'invalid',
 6: 'mixed_forest'}

In [13]:
#legend_list =  colOptions["legend"].toList(colOptions["legend"].size())
#landcover = legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

In [14]:
#landcover2 = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network'))

In [15]:
#landcover.getInfo()

In [16]:
#landcover2.getInfo()

In [17]:
#legend_list.getInfo() 

In [14]:
input_collection =  ee.ImageCollection(colOptions['name']) \
                      .filterBounds(mapBounds) \
                      .filterDate(startDate, endDate) \
                      .filterMetadata(colOptions["Cloudcover"],'less_than',maxCloudcover) \
                      .limit(5000) \
                      .map(lambda image: image.clip(mapBounds)) \
                      .map(lambda image: tools.MaskClear(image))  \
                      .map(lambda image: eoImage.attach_Date(image)) \
                      .map(lambda image: eoImage.attach_LonLat(image)) \
                      .map(lambda image: tools.addGeometry(colOptions,image)) 

s2_cloudless_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterBounds(mapBounds).filterDate(startDate, endDate)
projection = input_collection.first().select(netOptions["inputBands"][3]).projection()

input_collection = input_collection.map( lambda image: image.setDefaultProjection(crs=image.select(image.bandNames().slice(0,1)).projection()) \
                                                                     .reduceResolution(reducer= ee.Reducer.mean(),maxPixels=1024).reproject(crs=projection,scale=inputScaleSize))
s2_cloudless_col = s2_cloudless_col.map( lambda image: image.setDefaultProjection(crs=image.select(image.bandNames().slice(0,1)).projection()) \
                                                                    .reduceResolution(reducer= ee.Reducer.mean(),maxPixels=1024).reproject(crs=projection,scale=inputScaleSize)
                                                                    .rename('s2'))

s2_cloudless_col = s2_cloudless_col.map( lambda image: image.setDefaultProjection(crs=image.select(image.bandNames().slice(0,1)).projection()) \
                                                                    .reduceResolution(reducer= ee.Reducer.mean(),maxPixels=1024).reproject(crs=projection,scale=inputScaleSize))

input_collection  =  input_collection.map(lambda image: tools.MaskLand(image)).map(lambda image: \
                                        toolsUtils.scaleBands(netOptions["inputBands"],netOptions["inputScaling"],netOptions["inputOffset"],image)) \
                                                 .map(lambda image: toolsUtils.invalidInput(colOptions["sl2pDomain"],netOptions["inputBands"],image)) 

#input_collection=input_collection.combine(s2_cloudless_col)
partition = (colOptions["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition');  
input_collection  =  input_collection.map(lambda image: image.addBands(partition))#.combine(landcover)

In [16]:
partition.getInfo()

{'type': 'Image',
 'bands': [{'id': 'partition',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [1, 1],
   'origin': [-96, 39],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:footprint': {'type': 'Point',
   'coordinates': [-95.190753379558, 39.04015655639038]}}}

In [40]:
lc=partition.sample(region=site.geometry(), projection=projection, scale=30,geometries=True, dropNulls = True, factor=0.99)

In [54]:
l=lc.getInfo()

5

In [53]:
sampleList= ee.List(productCollection.first().bandNames().map(lambda bandName: ee.Dictionary({ 'bandName': bandName, 'data': sampleData.aggregate_array(bandName)})))

EEException: Required argument (image2) missing to function: Image.first

In [42]:
l=lc.getInfo()

In [45]:
l['features'][0]['properties']

{'partition': 5}

In [48]:
sampleList = ee.Dictionary(ee.Feature(partition).toDictionary()).getInfo()

In [50]:
partition.getInfo()

{'type': 'Image',
 'bands': [{'id': 'partition',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [1, 1],
   'origin': [-96, 39],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:footprint': {'type': 'Point',
   'coordinates': [-95.190753379558, 39.04015655639038]}}}

In [36]:
site1 = ee.Feature(site).buffer(20)
A=partition.sample(region=site1.geometry(), projection=projection, scale=30,geometries=True, dropNulls = True, factor=0.99)
A.getInfo()

{'type': 'FeatureCollection',
 'columns': {'partition': 'Short<0, 255>'},
 'properties': {'band_order': ['partition']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-95.19077248353427, 39.04023201503832]},
   'id': '0',
   'properties': {'partition': 5}}]}

In [39]:
site1 = ee.Feature(site).buffer(200)
A=partition.sample(region=site1.geometry(), projection=projection, scale=600,geometries=True, dropNulls = True, factor=0.99)
A.getInfo()

{'type': 'FeatureCollection',
 'columns': {'partition': 'Short<0, 255>'},
 'properties': {'band_order': ['partition']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-95.18956426882872, 39.04038986135921]},
   'id': '0',
   'properties': {'partition': 5}}]}

In [30]:
B=A.getInfo()

In [31]:
B['features']

[{'type': 'Feature',
  'geometry': {'geodesic': False,
   'type': 'Point',
   'coordinates': [-95.19077248353427, 39.04023201503832]},
  'id': '0',
  'properties': {'partition': 5}}]

In [ ]:
s.getInfo()

In [268]:
#lc=partition.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99)

In [269]:
# img = input_collection.first()
# SR =img.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99)
# SR.getInfo()

{'type': 'FeatureCollection',
 'columns': {'AOT': 'Float<0.0, 65535.0>',
  'B1': 'Float<0.0, 65535.0>',
  'B11': 'Float<0.0, 6.5535000000000005>',
  'B12': 'Float<0.0, 6.5535000000000005>',
  'B2': 'Float<0.0, 65535.0>',
  'B3': 'Float<0.0, 6.5535000000000005>',
  'B4': 'Float<0.0, 6.5535000000000005>',
  'B5': 'Float<0.0, 6.5535000000000005>',
  'B6': 'Float<0.0, 6.5535000000000005>',
  'B7': 'Float<0.0, 6.5535000000000005>',
  'B8': 'Float<0.0, 65535.0>',
  'B8A': 'Float<0.0, 6.5535000000000005>',
  'B9': 'Float<0.0, 65535.0>',
  'MSK_CLDPRB': 'Float<0.0, 255.0>',
  'MSK_SNWPRB': 'Float<0.0, 255.0>',
  'QA10': 'Float<0.0, 65535.0>',
  'QA20': 'Float<0.0, 4.294967295E9>',
  'QA60': 'Float<0.0, 65535.0>',
  'QC': 'Byte<0, 1>',
  'SCL': 'Float<0.0, 255.0>',
  'TCI_B': 'Float<0.0, 255.0>',
  'TCI_G': 'Float<0.0, 255.0>',
  'TCI_R': 'Float<0.0, 255.0>',
  'WVP': 'Float<0.0, 65535.0>',
  'cosRAA': 'Float<-3.2768, 3.2767>',
  'cosSZA': 'Float<0.0, 6.5535000000000005>',
  'cosVZA': 'Float<0.

In [262]:
landcover.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'version': 1618598763605530.0,
 'id': 'users/rfernand387/Legend_prosail_ccrs_big_clumped',
 'properties': {'system:asset_size': 12817},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [0, 0]},
   'id': '00000000000000000006',
   'properties': {'SL2P Network': 'deciduous_broadleaf_forest'}}]}

In [265]:
# legend_list = L.toList(legend.size())
# networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network'))

In [251]:
ee.Feature(L).get('SL2P Network').getInfo()

In [252]:


    # landcover = legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

    # # get corresponding networkIDs
    # networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \

In [283]:
landcover.getInfo()

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [282]:
legend_list.getInfo()

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [212]:
input_collection = input_collection.combine(S)

In [197]:
S=(colOptions["partition"]).filterBounds(mapBounds).map(lambda image: image.clip(mapBounds))

In [201]:
input_collection=input_collection.combine(S)

In [222]:
A=colOptions["legend"].getInfo()

In [229]:
[ff['properties']['SL2P Network'] for ff in A['features']]

['evergreen_needleaf_forest',
 'deciduous_broadleaf_forest',
 'deciduous_broadleaf_forest',
 'invalid',
 'shrublands',
 'polar_shrubland',
 'deciduous_broadleaf_forest',
 'polar_shrubland',
 'grassland_pasture',
 'close_cropland',
 'lichen_feathermoss',
 'evergreen_needleaf_forest',
 'grassland_pasture',
 'polar_grassland',
 'sphagnum_feathermoss',
 'invalid',
 'sparse_cropland',
 'invalid',
 'mixed_forest']

In [ ]:
L=colOptions["legend"].filter(ee.Filter.eq('Value', 5))
(colOptions["partition"]).filterBounds(mapBounds).mosaic().clip(mapBounds).rename('partition').getInfo()

In [75]:
SR =partition.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99)

In [77]:
input_collection.get('partition').getInfo()

In [27]:
#partition.getInfo()

In [67]:
PP=ee.FeatureCollection(partition)

In [68]:
PP_list=PP.toList(PP.size())

In [81]:
idx=partition.remap(legend,networkIDs, 0)

In [115]:
image.get('partition').getInfo()

In [153]:
img=input_collection.first()

In [159]:
img.get('partition').getInfo()

In [165]:
Network_Ind = ee.FeatureCollection(colOptions["Network_Ind"])
image = ee.Image(partition)
legend = ee.FeatureCollection(colOptions["legend"])

legend_list = legend.toList(legend.size())
landcover = legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))
networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
                                    .map(lambda propertyValue: ee.Feature(ee.FeatureCollection(Network_Ind).first()).toDictionary().getNumber(propertyValue))
idx=image.remap(landcover, networkIDs, 0).rename('networkID')

In [166]:
landcover.getInfo().to

[1, 3, 4, 17, 7, 8, 5, 11, 9, 15, 13, 2, 14, 10, 12, 18, 16, 19, 6]

In [167]:
networkIDs.getInfo()

['evergreen_needleaf_forest',
 'deciduous_broadleaf_forest',
 'deciduous_broadleaf_forest',
 'invalid',
 'shrublands',
 'polar_shrubland',
 'deciduous_broadleaf_forest',
 'polar_shrubland',
 'grassland_pasture',
 'close_cropland',
 'lichen_feathermoss',
 'evergreen_needleaf_forest',
 'grassland_pasture',
 'polar_grassland',
 'sphagnum_feathermoss',
 'invalid',
 'sparse_cropland',
 'invalid',
 'mixed_forest']

In [130]:
landcover.getInfo()

[1, 3, 4, 17, 7, 8, 5, 11, 9, 15, 13, 2, 14, 10, 12, 18, 16, 19, 6]

In [192]:
#networkIDs.getInfo()

In [122]:
image.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99).getInfo()

{'type': 'FeatureCollection',
 'columns': {'partition': 'Short<0, 255>'},
 'properties': {'band_order': ['partition']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-95.19077248353427, 39.04023201503832]},
   'id': '0',
   'properties': {'partition': 5}}]}

In [88]:
idx.getInfo()

{'type': 'Image',
 'bands': [{'id': 'networkID',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 11},
   'dimensions': [1, 1],
   'origin': [-96, 39],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'system:footprint': {'type': 'Point',
   'coordinates': [-95.190753379558, 39.04015655639038]}}}

In [89]:
SR =idx.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99)

In [90]:
SR.getInfo()

{'type': 'FeatureCollection',
 'columns': {'networkID': 'Byte<0, 11>'},
 'properties': {'band_order': ['networkID']},
 'features': [{'type': 'Feature',
   'geometry': {'geodesic': False,
    'type': 'Point',
    'coordinates': [-95.19077248353427, 39.04023201503832]},
   'id': '0',
   'properties': {'networkID': 2}}]}

In [48]:
Network_Ind.getInfo()

{'type': 'FeatureCollection',
 'columns': {'close_cropland': 'Integer',
  'deciduous_broadleaf_forest': 'Integer',
  'evergreen_needleaf_forest': 'Integer',
  'grassland_pasture': 'Integer',
  'invalid': 'Integer',
  'lichen_feathermoss': 'Integer',
  'lon': 'Integer',
  'mixed_forest': 'Integer',
  'polar_grassland': 'Integer',
  'polar_shrubland': 'Integer',
  'shrublands': 'Integer',
  'sparse_cropland': 'Integer',
  'sphagnum_feathermoss': 'Integer',
  'system:index': 'String'},
 'version': 1618611388723417,
 'id': 'users/rfernand387/Parameter_file_prosail_ccrs_big_clumped2',
 'properties': {'system:asset_size': 10991},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [0, 0]},
   'id': '00000000000000000000',
   'properties': {'close_cropland': 1,
    'deciduous_broadleaf_forest': 2,
    'evergreen_needleaf_forest': 3,
    'grassland_pasture': 4,
    'invalid': 0,
    'lichen_feathermoss': 5,
    'lon': 0,
    'mixed_forest': 11,
    'polar_grasslan

In [132]:
filtered = legend.filter(ee.Filter.eq('Value', 5))

In [133]:
filtered.getInfo()

{'type': 'FeatureCollection',
 'columns': {'B': 'Integer',
  'B (hex)': 'String',
  'Description': 'String',
  'Earth Engine Palette': 'String',
  'G': 'Integer',
  'G (hex)': 'String',
  'Lon': 'Integer',
  'R': 'Integer',
  'R (hex)': 'String',
  'SL2P Network': 'String',
  'Value': 'Integer',
  'system:index': 'String'},
 'version': 1618598763605530.0,
 'id': 'users/rfernand387/Legend_prosail_ccrs_big_clumped',
 'properties': {'system:asset_size': 12817},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [0, 0]},
   'id': '00000000000000000006',
   'properties': {'B': 61,
    'B (hex)': '3D',
    'Description': 'Temperate or sub-polar broadleaf deciduous forest',
    'Earth Engine Palette': "'148C3D' , // Temperate or sub-polar broadleaf deciduous forest",
    'G': 140,
    'G (hex)': '8C',
    'Lon': 0,
    'R': 20,
    'R (hex)': '14',
    'SL2P Network': 'deciduous_broadleaf_forest',
    'Value': 5}}]}

In [121]:
def makeIndexLayer(image, legend, Network_Ind):
    image = ee.Image(image)                          # partition image
    legend = ee.FeatureCollection(legend)            # legend to convert partition numbers to networks
    Network_Ind = ee.FeatureCollection(Network_Ind)  # legend to convert networks to networkIDs
    
    # get lists of valid partitions
    legend_list = legend.toList(legend.size())
    landcover = legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

    # get corresponding networkIDs
    networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
                                    .map(lambda propertyValue: ee.Feature(ee.FeatureCollection(Network_Ind).first()) \
                                    .toDictionary().getNumber(propertyValue))
    
    return image.remap(landcover, networkIDs, 0).rename('networkID')

In [122]:
# colOptions["legend"].get(0).getInfo()

In [67]:
#input_collection  =  input_collection.map(lambda image: image.addBands(makeIndexLayer(partition,colOptions["legend"],colOptions["Network_Ind"])))

In [70]:
#imageInput = imageInput.addBands(makeIndexLayer(partition,colOptions["legend"],colOptions["Network_Ind"]))

In [71]:
img = input_collection.first()
netList,imageInput,estimateSL2P = wrapperNNets(SL2P,partition, netOptions, colOptions,"estimate",variable,img)  

In [72]:
SR =img.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99)
var =estimateSL2P.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99)
part=partition.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99)

In [73]:
SR.getInfo()

{'type': 'FeatureCollection',
 'columns': {'AOT': 'Float<0.0, 65535.0>',
  'B1': 'Float<0.0, 65535.0>',
  'B11': 'Float<0.0, 6.5535000000000005>',
  'B12': 'Float<0.0, 6.5535000000000005>',
  'B2': 'Float<0.0, 65535.0>',
  'B3': 'Float<0.0, 6.5535000000000005>',
  'B4': 'Float<0.0, 6.5535000000000005>',
  'B5': 'Float<0.0, 6.5535000000000005>',
  'B6': 'Float<0.0, 6.5535000000000005>',
  'B7': 'Float<0.0, 6.5535000000000005>',
  'B8': 'Float<0.0, 65535.0>',
  'B8A': 'Float<0.0, 6.5535000000000005>',
  'B9': 'Float<0.0, 65535.0>',
  'MSK_CLDPRB': 'Float<0.0, 255.0>',
  'MSK_SNWPRB': 'Float<0.0, 255.0>',
  'QA10': 'Float<0.0, 65535.0>',
  'QA20': 'Float<0.0, 4.294967295E9>',
  'QA60': 'Float<0.0, 65535.0>',
  'QC': 'Byte<0, 1>',
  'SCL': 'Float<0.0, 255.0>',
  'TCI_B': 'Float<0.0, 255.0>',
  'TCI_G': 'Float<0.0, 255.0>',
  'TCI_R': 'Float<0.0, 255.0>',
  'WVP': 'Float<0.0, 65535.0>',
  'cosRAA': 'Float<-3.2768, 3.2767>',
  'cosSZA': 'Float<0.0, 6.5535000000000005>',
  'cosVZA': 'Float<0.

In [23]:
1

1

In [ ]:
# def wrapperNNets(network, partition, netOptions, colOptions, suffixName, outputName, imageInput):
#     # typecast function parameters
#     network = ee.List(network)
#     partition = ee.Image(partition)
#     netOptions = netOptions
#     colOptions = colOptions
#     suffixName = suffixName
#     imageInput = ee.Image(imageInput)
#     outputName = outputName
    
#     # parse partition  used to identify network to use
#     partition = partition.clip(imageInput.geometry()).select(['partition'])

#     # determine networks based on collection
#     netList = ee.List(network.get(ee.Number(netOptions.get("variable")).subtract(1)))

#     # parse land cover into network index and add to input image
#     imageInput = imageInput.addBands(makeIndexLayer(partition,colOptions["legend"],colOptions["Network_Ind"]))

In [ ]:
def makeIndexLayer_2(image, legend, Network_Ind):
    image = ee.Image(image)                          # partition image
    legend = ee.FeatureCollection(legend)            # legend to convert partition numbers to networks
    Network_Ind = ee.FeatureCollection(Network_Ind)  # legend to convert networks to networkIDs
    
    # get lists of valid partitions
    legend_list = legend.toList(legend.size())
    landcover = legend_list.map(lambda feature: ee.Feature(feature).getNumber('Value'))

    # get corresponding networkIDs
    networkIDs = legend_list.map(lambda feature: ee.Feature(feature).get('SL2P Network')) \
                                    .map(lambda propertyValue: ee.Feature(ee.FeatureCollection(Network_Ind).first()) \
                                    .toDictionary().getNumber(propertyValue))
    
    return image.remap(landcover, networkIDs, 0).rename('networkID')

In [116]:
part.getInfo()['features'][0]['properties'],SR.getInfo()['features'][0]['properties'],var.getInfo()['features'][0]['properties'],var_unc.getInfo()['features'][0]['properties']

({'partition': 5},
 {'QA_PIXEL': 21824,
  'QA_RADSAT': 0,
  'QC': 0,
  'SR_B1': 7829,
  'SR_B2': 7965,
  'SR_B3': 0.04274249999999999,
  'SR_B4': 0.02855250000000001,
  'SR_B5': 0.2790225,
  'SR_B6': 0.146885,
  'SR_B7': 0.06036999999999998,
  'SR_QA_AEROSOL': 96,
  'ST_ATRAN': 7076,
  'ST_B10': 45405,
  'ST_CDIST': 676,
  'ST_DRAD': 1141,
  'ST_EMIS': 9856,
  'ST_EMSD': 39,
  'ST_QA': 220,
  'ST_TRAD': 9569,
  'ST_URAD': 2428,
  'cosRAA': 0,
  'cosSZA': 0.7868,
  'cosVZA': 0.9996,
  'date': 1662742872619,
  'latitude': 39.040100199097424,
  'longitude': -95.19059511271381},
 {'estimateLAI': 1.9475146135165375, 'networkID': 2, 'partition': 5},
 {'errorLAI': 0.5177120606372808, 'networkID': 2, 'partition': 5})

In [119]:
len(netList.getInfo())

12

In [122]:
I=imageInput.sample(region=site.geometry(), projection=projection, scale=outputScaleSize,geometries=True, dropNulls = True, factor=0.99)

In [124]:
I.getInfo()['features']

[{'type': 'Feature',
  'geometry': {'geodesic': False,
   'type': 'Point',
   'coordinates': [-95.19059511271381, 39.040100199097424]},
  'id': '0',
  'properties': {'QA_PIXEL': 21824,
   'QA_RADSAT': 0,
   'QC': 0,
   'SR_B1': 7829,
   'SR_B2': 7965,
   'SR_B3': 0.04274249999999999,
   'SR_B4': 0.02855250000000001,
   'SR_B5': 0.2790225,
   'SR_B6': 0.146885,
   'SR_B7': 0.06036999999999998,
   'SR_QA_AEROSOL': 96,
   'ST_ATRAN': 7076,
   'ST_B10': 45405,
   'ST_CDIST': 676,
   'ST_DRAD': 1141,
   'ST_EMIS': 9856,
   'ST_EMSD': 39,
   'ST_QA': 220,
   'ST_TRAD': 9569,
   'ST_URAD': 2428,
   'cosRAA': 0,
   'cosSZA': 0.7868,
   'cosVZA': 0.9996,
   'date': 1662742872619,
   'latitude': 39.040100199097424,
   'longitude': -95.19059511271381,
   'networkID': 2}}]